# Uber - Eats Delivery Partner Route Performance

```SQL
CREATE TABLE fct_delivery_routes (
    route_id INTEGER,
    delivery_partner_id INTEGER,
    pickup_count INTEGER,
    delivery_time FLOAT,
    earnings FLOAT,
    route_date DATE
);

INSERT INTO fct_delivery_routes (route_id, delivery_partner_id, pickup_count, delivery_time, earnings, route_date)
VALUES
    (101, 1001, 1, 15.5, 8.25, '2024-10-05'),
    (102, 1002, 2, 20.0, 10.0, '2024-10-10'),
    (103, 1003, 3, 25.0, 12.5, '2024-10-15'),
    (104, 1001, 2, 18.0, 9.0, '2024-10-20'),
    (105, 1002, 1, 16.0, 8.0, '2024-10-25'),
    (106, 1003, 4, 30.0, 15.0, '2024-11-05'),
    (107, 1001, 3, 28.0, 14.0, '2024-11-10'),
    (108, 1002, 2, 22.0, 10.0, '2024-11-15'),
    (109, 1003, 1, 17.0, 8.5, '2024-11-20'),
    (110, 1001, 3, 27.5, 13.0, '2024-12-05'),
    (111, 1002, 2, 20.5, 10.5, '2024-12-10'),
    (112, 1003, 1, 16.5, 7.5, '2024-12-15'),
    (113, 1001, 4, 32.0, 11.0, '2024-12-20'),
    (114, 1002, 2, 21.0, 11.0, '2024-12-25'),
    (115, 1003, 3, 26.0, 12.0, '2024-12-30');

SELECT * FROM fct_delivery_routes;
```

In [1]:
import pandas as pd
import numpy as np

In [3]:
df_delivery = pd.read_csv('Data/029/fct_delivery_routes.csv', parse_dates=['route_date'])

df_delivery.head()

,route_id,delivery_partner_id,pickup_count,delivery_time,earnings,route_date
0,101,1001,1,15.5,8.25,2024-10-05
1,102,1002,2,20.0,10.00,2024-10-10
2,103,1003,3,25.0,12.50,2024-10-15
3,104,1001,2,18.0,9.00,2024-10-20
4,105,1002,1,16.0,8.00,2024-10-25


# Pregunta 1

### Para todas las rutas de entrega entre el 1 de octubre y el 31 de diciembre de 2024, ¿qué porcentaje de las rutas tuvo recolecciones múltiples (es decir, 2 o más)? Esta métrica cuantificará con qué frecuencia ocurre el agrupamiento de pedidos para ayudar a evaluar la eficiencia de las rutas.

In [4]:
# 1. Filtramos por el rango de fechas
df_q4 = df_delivery[df_delivery['route_date'].between('2024-10-01', '2024-12-31')]

# 2. Calculamos el porcentaje usando una máscara booleana
# La media de una serie de booleanos (True/False) nos da la proporción
porcentaje_bundling = (df_q4['pickup_count'] >= 2).mean() * 100

print(f"El porcentaje de agrupamiento fue: {porcentaje_bundling:.2f}%")

El porcentaje de agrupamiento fue: 73.33%


```SQL
SELECT 
    COUNT(CASE WHEN pickup_count >= 2 THEN 1 END) * 100.0 / COUNT(*) AS bundling_percentage
FROM fct_delivery_routes
WHERE route_date BETWEEN '2024-10-01' AND '2024-12-31';
```

# Pregunta 2

### Para las rutas de entrega con recolecciones múltiples entre el 1 de octubre y el 31 de diciembre de 2024, ¿cómo difiere el tiempo promedio de entrega entre las rutas con exactamente 2 pedidos y las rutas con 3 o más pedidos? Utilice una sentencia CASE para segmentar las rutas en consecuencia. Este análisis aclarará el impacto de los diferentes niveles de agrupación de pedidos en el rendimiento de la entrega.

In [5]:
# 1. Filtramos el periodo y solo rutas múltiples
df_q4_multi = df_delivery[
    (df_delivery['route_date'].between('2024-10-01', '2024-12-31')) & 
    (df_delivery['pickup_count'] >= 2)
].copy()

# 2. Creamos los segmentos
df_q4_multi['cluster_size'] = np.where(
    df_q4_multi['pickup_count'] == 2, 
    'Exactly 2 Orders', 
    '3 or More Orders'
)

# 3. Calculamos el promedio por segmento
analisis_tiempo = df_q4_multi.groupby('cluster_size')['delivery_time'].mean()

print(analisis_tiempo)

cluster_size
3 or More Orders    28.083333
Exactly 2 Orders    20.300000
Name: delivery_time, dtype: float64


```SQL
SELECT 
    CASE 
        WHEN pickup_count = 2 THEN 'Exactly 2 Orders'
        WHEN pickup_count >= 3 THEN '3 or More Orders'
    END AS cluster_size,
    AVG(delivery_time) AS avg_delivery_time
FROM fct_delivery_routes
WHERE route_date BETWEEN '2024-10-01' AND '2024-12-31'
  AND pickup_count >= 2
GROUP BY cluster_size;
```

# Pregunta 3

### ¿Cuál es el promedio de ganancias por recolección (earnings per pickup) en todas las rutas?

### Nota: Algunas filas tienen valores faltantes en la columna de ganancias. Antes de calcular el valor final, reemplace cualquier ganancia faltante con el valor promedio de las ganancias.

In [6]:
# 1. Calculamos el promedio global de ganancias
ganancia_promedio = df_delivery['earnings'].mean()

# 2. Rellenamos los valores faltantes (imputación)
df_delivery['earnings_filled'] = df_delivery['earnings'].fillna(ganancia_promedio)

# 3. Calculamos la métrica por fila y luego su promedio global
df_delivery['earnings_per_pickup'] = df_delivery['earnings_filled'] / df_delivery['pickup_count']
resultado_final = df_delivery['earnings_per_pickup'].mean()

print(f"Ganancia promedio por pickup: {resultado_final:.2f}")

Ganancia promedio por pickup: 5.41


```SQL
WITH avg_table AS (
    SELECT AVG(earnings) AS global_avg FROM fct_delivery_routes
)
SELECT 
    AVG(COALESCE(earnings, (SELECT global_avg FROM avg_table)) / pickup_count) AS avg_earnings_per_pickup
FROM fct_delivery_routes;
```